<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/CartanBlock2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================
# Block Calculus Engine + Chern–Simons (3D, stable, no diffgeom)
# Gauge connection A -> curvature F -> CS 3-form and identities
# Google Colab ready
# ============================================================

from sympy import symbols, Function, Matrix, diff, simplify

# ------------------------------------------------------------
# 1. Coordinates (3D)
# ------------------------------------------------------------
x, y, z = symbols('x y z', real=True)

# ------------------------------------------------------------
# 2. Formal basis 1-forms and volume 3-form
# ------------------------------------------------------------
dx, dy, dz, dV = symbols('dx dy dz dV')  # dV will represent dx∧dy∧dz

# Sign conventions:
# dx∧dy∧dz = +dV, and wedge is totally antisymmetric.

def eps_sign(a, b, c):
    """Return sign of permutation (a,b,c) relative to (dx,dy,dz); 0 if any repeated."""
    forms = [a,b,c]
    if len(set(forms)) < 3:
        return 0
    base = [dx, dy, dz]
    idx = [base.index(f) for f in forms]
    inv = 0
    for i in range(3):
        for j in range(i+1,3):
            if idx[i] > idx[j]:
                inv += 1
    return -1 if inv % 2 else 1

def wedge_1_1(a, b):
    """Wedge of basis 1-forms -> symbolic 2-form token (unordered pair + sign handled later)."""
    if a == b:
        return 0
    return (a, b)  # represent 2-form basis as ordered pair

def wedge_2_1(twoform, oneform):
    """Wedge of a represented 2-form (a,b) with 1-form c -> ± dV."""
    if twoform == 0:
        return 0
    a,b = twoform
    s = eps_sign(a,b,oneform)
    return s * dV

def wedge_1_2(oneform, twoform):
    """Wedge of 1-form with 2-form -> ± dV."""
    if twoform == 0:
        return 0
    a,b = twoform
    s = eps_sign(oneform,a,b)
    return s * dV

# ------------------------------------------------------------
# 3. Exterior derivative of scalar functions
# ------------------------------------------------------------
def d_scalar(f):
    return (diff(f,x), diff(f,y), diff(f,z))  # coefficients along dx,dy,dz

# ------------------------------------------------------------
# 4. Build curvature components from user-defined Ax,Ay,Az
#    F_{ij} = ∂i Aj - ∂j Ai + [Ai,Aj]
# ------------------------------------------------------------
def curvature_components(Ax, Ay, Az):
    # partials of matrices
    def dmat(M, var):
        return Matrix([[diff(M[i,j], var) for j in range(M.shape[1])]
                       for i in range(M.shape[0])])

    Fxy = simplify(dmat(Ay, x) - dmat(Ax, y) + (Ax*Ay - Ay*Ax))
    Fyz = simplify(dmat(Az, y) - dmat(Ay, z) + (Ay*Az - Az*Ay))
    Fzx = simplify(dmat(Ax, z) - dmat(Az, x) + (Az*Ax - Ax*Az))
    return Fxy, Fyz, Fzx

# ------------------------------------------------------------
# 5. Chern–Simons 3-form density
#    L_CS = Tr(A∧dA + 2/3 A∧A∧A)
#
# In components (3D):
#   A = Ax dx + Ay dy + Az dz
#   dA contributes (∂i Aj - ∂j Ai) dx^i∧dx^j
#   A∧dA -> Tr( Ax*(∂y Az-∂z Ay) + Ay*(∂z Ax-∂x Az) + Az*(∂x Ay-∂y Ax) ) dV
#   A∧A∧A -> Tr( Ax*Ay*Az + Ay*Az*Ax + Az*Ax*Ay - Ax*Az*Ay - Ay*Ax*Az - Az*Ay*Ax ) dV
# ------------------------------------------------------------
def trace(M):
    return sum(M[i,i] for i in range(M.shape[0]))

def cs_density(Ax, Ay, Az):
    # dA (abelian part) curls:
    def dmat(M, var):
        return Matrix([[diff(M[i,j], var) for j in range(M.shape[1])]
                       for i in range(M.shape[0])])

    curl_x = dmat(Az, y) - dmat(Ay, z)   # (∂y Az - ∂z Ay)
    curl_y = dmat(Ax, z) - dmat(Az, x)   # (∂z Ax - ∂x Az)
    curl_z = dmat(Ay, x) - dmat(Ax, y)   # (∂x Ay - ∂y Ax)

    term_AdA = simplify(trace(Ax*curl_x + Ay*curl_y + Az*curl_z))

    # Totally antisymmetrized triple product (non-abelian):
    triple = (Ax*Ay*Az + Ay*Az*Ax + Az*Ax*Ay
              - Ax*Az*Ay - Ay*Ax*Az - Az*Ay*Ax)
    term_AAA = simplify(trace(triple))

    # L_CS density coefficient multiplying dV:
    Lcs = simplify(term_AdA + (2/3)*term_AAA)
    return Lcs

# ------------------------------------------------------------
# 6. Useful identity: d L_CS = Tr(F∧F) (formal check)
# In 3D, F is a 2-form and F∧F is a 4-form (formally zero in 3D),
# but the algebraic structure is still meaningful for extending to 4D.
# Here we compute Tr(Fxy*Fyz + cyclic) as a proxy for the 4D Pontryagin density.
# ------------------------------------------------------------
def pontryagin_proxy(Fxy, Fyz, Fzx):
    # proxy ~ Tr(Fxy*Fyz + Fyz*Fzx + Fzx*Fxy)
    return simplify(trace(Fxy*Fyz + Fyz*Fzx + Fzx*Fxy))

# ============================================================
# USER INPUT SECTION
# ============================================================

print("=== USER INPUT (choose an example) ===")

# ---------- Example A: Abelian U(1) embedded as 1x1 ----------
# Comment out Example B if you use this.
a = Function('a')(x,y,z)
Ax = Matrix([[0]])
Ay = Matrix([[-z]])  # e.g. A = -z dy  => F = -dz∧dy
Az = Matrix([[0]])

print("\nExample A (Abelian 1x1):")
print("Ax,Ay,Az ="); display(Ax); display(Ay); display(Az)

# ---------- Example B: Simple non-abelian 2x2 (turn on if desired) ----------
# alpha, beta = symbols('alpha beta', real=True)
# Ax = Matrix([[0, alpha*z],
#              [0, 0]])
# Ay = Matrix([[0, 0],
#              [beta*z, 0]])
# Az = Matrix([[0, 0],
#              [0, 0]])
# print("\nExample B (Non-abelian 2x2):")
# print("Ax,Ay,Az ="); display(Ax); display(Ay); display(Az)

# ============================================================
# COMPUTATIONS
# ============================================================

Fxy, Fyz, Fzx = curvature_components(Ax, Ay, Az)

print("\nCurvature components:")
print("Fxy ="); display(Fxy)
print("Fyz ="); display(Fyz)
print("Fzx ="); display(Fzx)

Lcs = cs_density(Ax, Ay, Az)

print("\nChern–Simons 3-form density L_CS (coefficient of dV):")
display(Lcs)
print("\nSo  L_CS = Lcs * dV")

Pproxy = pontryagin_proxy(Fxy, Fyz, Fzx)
print("\nPontryagin proxy (useful when extending to 4D):")
display(Pproxy)

print("""
NOTES (what is 'useful' here):

1) Variational meaning:
   In 3D pure Chern–Simons, δS/δA ~ F  -> equations of motion: F = 0.
   This notebook gives F explicitly for any user-entered Ax,Ay,Az.

2) Boundary physics:
   Changes in L_CS under gauge transformations become boundary terms
   (source of edge currents / WZW in many contexts).

3) Quantities you can explore right away:
   - For abelian cases: Lcs reduces to A·(curl A) (magnetic helicity density).
   - For non-abelian cases: the extra (2/3) Tr(A∧A∧A) term is visible and computable.

Next upgrades (if you want):
   • add gauge transformation A -> g^{-1}Ag + g^{-1}dg and compute ΔL_CS
   • implement Wilson loop / holonomy approximation from Ax,Ay,Az
   • specialize to su(2) with Pauli matrices basis, compute level quantization hints
""")


=== USER INPUT (choose an example) ===

Example A (Abelian 1x1):
Ax,Ay,Az =


Matrix([[0]])

Matrix([[-z]])

Matrix([[0]])


Curvature components:
Fxy =


...


Chern–Simons 3-form density L_CS (coefficient of dV):


0


So  L_CS = Lcs * dV

Pontryagin proxy (useful when extending to 4D):


0


NOTES (what is 'useful' here):

1) Variational meaning:
   In 3D pure Chern–Simons, δS/δA ~ F  -> equations of motion: F = 0.
   This notebook gives F explicitly for any user-entered Ax,Ay,Az.

2) Boundary physics:
   Changes in L_CS under gauge transformations become boundary terms
   (source of edge currents / WZW in many contexts).

3) Quantities you can explore right away:
   - For abelian cases: Lcs reduces to A·(curl A) (magnetic helicity density).
   - For non-abelian cases: the extra (2/3) Tr(A∧A∧A) term is visible and computable.

Next upgrades (if you want):
   • add gauge transformation A -> g^{-1}Ag + g^{-1}dg and compute ΔL_CS
   • implement Wilson loop / holonomy approximation from Ax,Ay,Az
   • specialize to su(2) with Pauli matrices basis, compute level quantization hints

